In [153]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import r2_score

%matplotlib inline

## Test for test.csv
_____

In [154]:
df_test = pd.read_csv('datasets/test.csv')
df_test

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type
0,2658,902301120,190,RM,69.0,9142,Pave,Grvl,Reg,Lvl,...,0,0,0,NaN,NaN,NaN,0,4,2006,WD
1,2718,905108090,90,RL,NaN,9662,Pave,NaN,IR1,Lvl,...,0,0,0,NaN,NaN,NaN,0,8,2006,WD
2,2414,528218130,60,RL,58.0,17104,Pave,NaN,IR1,Lvl,...,0,0,0,NaN,NaN,NaN,0,9,2006,New
3,1989,902207150,30,RM,60.0,8520,Pave,NaN,Reg,Lvl,...,0,0,0,NaN,NaN,NaN,0,7,2007,WD
4,625,535105100,20,RL,NaN,9500,Pave,NaN,IR1,Lvl,...,0,185,0,NaN,NaN,NaN,0,7,2009,WD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
873,1662,527377110,60,RL,80.0,8000,Pave,NaN,Reg,Lvl,...,0,0,0,NaN,NaN,NaN,0,11,2007,WD
874,1234,535126140,60,RL,90.0,14670,Pave,NaN,Reg,Lvl,...,0,0,0,NaN,MnPrv,NaN,0,8,2008,WD
875,1373,904100040,20,RL,55.0,8250,Pave,NaN,Reg,Lvl,...,0,0,0,NaN,NaN,NaN,0,8,2008,WD
876,1672,527425140,20,RL,60.0,9000,Pave,NaN,Reg,Lvl,...,0,0,0,NaN,GdWo,NaN,0,5,2007,WD


In [155]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 878 entries, 0 to 877
Data columns (total 80 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Id               878 non-null    int64  
 1   PID              878 non-null    int64  
 2   MS SubClass      878 non-null    int64  
 3   MS Zoning        878 non-null    object 
 4   Lot Frontage     718 non-null    float64
 5   Lot Area         878 non-null    int64  
 6   Street           878 non-null    object 
 7   Alley            58 non-null     object 
 8   Lot Shape        878 non-null    object 
 9   Land Contour     878 non-null    object 
 10  Utilities        878 non-null    object 
 11  Lot Config       878 non-null    object 
 12  Land Slope       878 non-null    object 
 13  Neighborhood     878 non-null    object 
 14  Condition 1      878 non-null    object 
 15  Condition 2      878 non-null    object 
 16  Bldg Type        878 non-null    object 
 17  House Style     

## Cleaning
---------

In [156]:
# lowercase all columns, then replace whitespace with underscore
df_test.columns = df_test.columns.str.lower().str.replace(' ','_')
df_test

,id,pid,ms_subclass,ms_zoning,lot_frontage,lot_area,street,alley,lot_shape,land_contour,...,3ssn_porch,screen_porch,pool_area,pool_qc,fence,misc_feature,misc_val,mo_sold,yr_sold,sale_type
0,2658,902301120,190,RM,69.0,9142,Pave,Grvl,Reg,Lvl,...,0,0,0,NaN,NaN,NaN,0,4,2006,WD
1,2718,905108090,90,RL,NaN,9662,Pave,NaN,IR1,Lvl,...,0,0,0,NaN,NaN,NaN,0,8,2006,WD
2,2414,528218130,60,RL,58.0,17104,Pave,NaN,IR1,Lvl,...,0,0,0,NaN,NaN,NaN,0,9,2006,New
3,1989,902207150,30,RM,60.0,8520,Pave,NaN,Reg,Lvl,...,0,0,0,NaN,NaN,NaN,0,7,2007,WD
4,625,535105100,20,RL,NaN,9500,Pave,NaN,IR1,Lvl,...,0,185,0,NaN,NaN,NaN,0,7,2009,WD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
873,1662,527377110,60,RL,80.0,8000,Pave,NaN,Reg,Lvl,...,0,0,0,NaN,NaN,NaN,0,11,2007,WD
874,1234,535126140,60,RL,90.0,14670,Pave,NaN,Reg,Lvl,...,0,0,0,NaN,MnPrv,NaN,0,8,2008,WD
875,1373,904100040,20,RL,55.0,8250,Pave,NaN,Reg,Lvl,...,0,0,0,NaN,NaN,NaN,0,8,2008,WD
876,1672,527425140,20,RL,60.0,9000,Pave,NaN,Reg,Lvl,...,0,0,0,NaN,GdWo,NaN,0,5,2007,WD


In [157]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 878 entries, 0 to 877
Data columns (total 80 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               878 non-null    int64  
 1   pid              878 non-null    int64  
 2   ms_subclass      878 non-null    int64  
 3   ms_zoning        878 non-null    object 
 4   lot_frontage     718 non-null    float64
 5   lot_area         878 non-null    int64  
 6   street           878 non-null    object 
 7   alley            58 non-null     object 
 8   lot_shape        878 non-null    object 
 9   land_contour     878 non-null    object 
 10  utilities        878 non-null    object 
 11  lot_config       878 non-null    object 
 12  land_slope       878 non-null    object 
 13  neighborhood     878 non-null    object 
 14  condition_1      878 non-null    object 
 15  condition_2      878 non-null    object 
 16  bldg_type        878 non-null    object 
 17  house_style     

In [158]:
# Create a new DataFrame with the columns & rows you are going to drop
df_test_dropped = pd.DataFrame(df_test, columns=['id', 'alley', 'pool_qc', 'misc_feature'])
df_test_dropped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 878 entries, 0 to 877
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            878 non-null    int64 
 1   alley         58 non-null     object
 2   pool_qc       4 non-null      object
 3   misc_feature  41 non-null     object
dtypes: int64(1), object(3)
memory usage: 27.6+ KB


In [159]:
df_test.loc[(df_test['ms_zoning']=='A (agr)') 
            | (df_test['ms_zoning']=='I (all)') 
            | (df_test['ms_zoning']=='C (all)'), ['ms_zoning']]

,ms_zoning
148,C (all)
270,C (all)
395,C (all)
399,C (all)
522,I (all)
757,C (all)
803,C (all)


In [160]:
df_test_0 = df_test[:].fillna(0)
df_test_0

,id,pid,ms_subclass,ms_zoning,lot_frontage,lot_area,street,alley,lot_shape,land_contour,...,3ssn_porch,screen_porch,pool_area,pool_qc,fence,misc_feature,misc_val,mo_sold,yr_sold,sale_type
0,2658,902301120,190,RM,69.0,9142,Pave,Grvl,Reg,Lvl,...,0,0,0,0,0,0,0,4,2006,WD
1,2718,905108090,90,RL,0.0,9662,Pave,0,IR1,Lvl,...,0,0,0,0,0,0,0,8,2006,WD
2,2414,528218130,60,RL,58.0,17104,Pave,0,IR1,Lvl,...,0,0,0,0,0,0,0,9,2006,New
3,1989,902207150,30,RM,60.0,8520,Pave,0,Reg,Lvl,...,0,0,0,0,0,0,0,7,2007,WD
4,625,535105100,20,RL,0.0,9500,Pave,0,IR1,Lvl,...,0,185,0,0,0,0,0,7,2009,WD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
873,1662,527377110,60,RL,80.0,8000,Pave,0,Reg,Lvl,...,0,0,0,0,0,0,0,11,2007,WD
874,1234,535126140,60,RL,90.0,14670,Pave,0,Reg,Lvl,...,0,0,0,0,MnPrv,0,0,8,2008,WD
875,1373,904100040,20,RL,55.0,8250,Pave,0,Reg,Lvl,...,0,0,0,0,0,0,0,8,2008,WD
876,1672,527425140,20,RL,60.0,9000,Pave,0,Reg,Lvl,...,0,0,0,0,GdWo,0,0,5,2007,WD


In [168]:
# create a new dataframe with only id column
df_test_1 = pd.DataFrame(df_test_0, columns=['id'])
df_test_1

,id
0,2658
1,2718
2,2414
3,1989
4,625
...,...
873,1662
874,1234
875,1373
876,1672


In [169]:
# export df_test_1 for future merge use
df_test_1.to_csv('datasets/df_test_1.csv', index=False)

In [17]:
# Drop all insignificant columns (pre converting to int.)
df_test = df_test.drop(columns = ['alley', 'pool_qc', 'misc_feature'])
df_test = df_test.drop(df_test[df_test['ms_zoning'] == 'A (agr)'].index)
df_test = df_test.drop(df_test[df_test['ms_zoning'] == 'C (all)'].index)
df_test = df_test.drop(df_test[df_test['ms_zoning'] == 'I (all)'].index)

In [22]:
df_test['ms_zoning'] = df_test['ms_zoning'].replace({
    'RH' : 4,
    'RL': 3,
    'RM': 2,
    'FV': 1
})
df_test['ms_zoning']

0      2
1      3
2      3
3      2
4      3
      ..
873    3
874    3
875    3
876    3
877    3
Name: ms_zoning, Length: 871, dtype: int64

In [23]:
df_test['exter_qual'] = df_test['exter_qual'].replace({
    'Ex' : 5,
    'Gd' : 4,
    'TA': 3,
    'Fa': 2,
    'Po':1
})
df_test['exter_qual']

0      3
1      3
2      4
3      4
4      3
      ..
873    3
874    4
875    3
876    3
877    3
Name: exter_qual, Length: 871, dtype: int64

In [24]:
df_test['exter_cond'] = df_test['exter_cond'].replace({
    'Ex' : 5,
    'Gd' : 4,
    'TA': 3,
    'Fa': 2,
    'Po': 1
})

df_test['exter_cond']

0      2
1      3
2      3
3      3
4      3
      ..
873    3
874    4
875    3
876    3
877    4
Name: exter_cond, Length: 871, dtype: int64

In [26]:
df_test['bsmt_qual'] = df_test['bsmt_qual'].replace({
    'Ex' : 5,
    'Gd' : 4,
    'TA': 3,
    'Fa': 2,
    'Po': 1,
    'NA' : 0
})
df_test['bsmt_qual']

0      2.0
1      4.0
2      4.0
3      3.0
4      4.0
      ... 
873    3.0
874    3.0
875    3.0
876    3.0
877    3.0
Name: bsmt_qual, Length: 871, dtype: float64

In [27]:
df_test['bsmt_cond'] = df_test['bsmt_cond'].replace({
    'Ex' : 5,
    'Gd' : 4,
    'TA': 3,
    'Fa': 2,
    'Po': 1,
    'NA' : 0
})
df_test['bsmt_cond']

0      3.0
1      3.0
2      4.0
3      3.0
4      3.0
      ... 
873    3.0
874    3.0
875    3.0
876    3.0
877    3.0
Name: bsmt_cond, Length: 871, dtype: float64

In [28]:
df_test['bsmt_exposure'] = df_test['bsmt_exposure'].replace({
    'Gd' : 4,
    'Av': 3,
    'Mn': 2,
    'No': 1,
    'NA' : 0
})
df_test['bsmt_exposure']

0      1.0
1      1.0
2      3.0
3      1.0
4      1.0
      ... 
873    1.0
874    1.0
875    1.0
876    1.0
877    1.0
Name: bsmt_exposure, Length: 871, dtype: float64

In [29]:
df_test['bsmtfin_type_1'] = df_test['bsmtfin_type_1'].replace({
    'GLQ' : 6,
    'ALQ': 5,
    'BLQ': 4,
    'Rec': 3,
    'LwQ' : 2,
    'Unf': 1,
    'NA': 0
})
df_test['bsmtfin_type_1']

0      1.0
1      1.0
2      6.0
3      1.0
4      4.0
      ... 
873    5.0
874    4.0
875    4.0
876    5.0
877    4.0
Name: bsmtfin_type_1, Length: 871, dtype: float64

In [30]:
df_test['bsmtfin_type_2'] = df_test['bsmtfin_type_2'].replace({
    'GLQ' : 6,
    'ALQ': 5,
    'BLQ': 4,
    'Rec': 3,
    'LwQ' : 2,
    'Unf': 1,
    'NA': 0
})
df_test['bsmtfin_type_2']

0      1.0
1      1.0
2      1.0
3      1.0
4      1.0
      ... 
873    2.0
874    1.0
875    2.0
876    1.0
877    1.0
Name: bsmtfin_type_2, Length: 871, dtype: float64

In [31]:
df_test['heating'] = df_test['heating'].replace({
    'Floor' : 6,
    'GasA': 5,
    'GasW': 4,
    'Grav': 3,
    'OthW' : 2,
    'Wall': 1,
})
df_test['heating']

0      5
1      5
2      5
3      5
4      5
      ..
873    5
874    5
875    5
876    5
877    5
Name: heating, Length: 871, dtype: int64

In [32]:
df_test['heating_qc'] = df_test['heating_qc'].replace({
    'Ex' : 5,
    'Gd': 4,
    'TA': 3,
    'Fa': 2,
    'Po' : 1,
})
df_test['heating_qc']

0      4
1      3
2      5
3      3
4      4
      ..
873    3
874    5
875    5
876    3
877    3
Name: heating_qc, Length: 871, dtype: int64

In [33]:
df_test['central_air'] = df_test['central_air'].replace({
    'N' : 0,
    'Y': 1,
})
df_test['central_air']

0      0
1      1
2      1
3      1
4      1
      ..
873    1
874    1
875    1
876    1
877    1
Name: central_air, Length: 871, dtype: int64

In [35]:
df_test['kitchen_qual'] = df_test['kitchen_qual'].replace({
    'Ex' : 5,
    'Gd': 4,
    'TA': 3,
    'Fa': 2,
    'Po' : 1,
})
df_test['kitchen_qual']

0      2
1      3
2      4
3      3
4      3
      ..
873    3
874    4
875    3
876    3
877    3
Name: kitchen_qual, Length: 871, dtype: int64

In [36]:
df_test['fireplace_qu'] = df_test['fireplace_qu'].replace({
    'Ex' : 5,
    'Gd': 4,
    'TA': 3,
    'Fa': 2,
    'Po' : 1,
    'NA': 0
})
df_test['fireplace_qu']

0      NaN
1      NaN
2      4.0
3      NaN
4      4.0
      ... 
873    3.0
874    4.0
875    3.0
876    NaN
877    3.0
Name: fireplace_qu, Length: 871, dtype: float64

In [37]:
df_test['garage_qual'] = df_test['garage_qual'].replace({
    'Ex' : 5,
    'Gd': 4,
    'TA': 3,
    'Fa': 2,
    'Po' : 1,
    'NA': 0
})
df_test['garage_qual']

0      1.0
1      3.0
2      3.0
3      2.0
4      3.0
      ... 
873    3.0
874    3.0
875    3.0
876    3.0
877    3.0
Name: garage_qual, Length: 871, dtype: float64

In [38]:
df_test['garage_cond'] = df_test['garage_cond'].replace({
    'Ex' : 5,
    'Gd': 4,
    'TA': 3,
    'Fa': 2,
    'Po' : 1,
    'NA': 0
})
df_test['garage_cond']

0      1.0
1      3.0
2      3.0
3      3.0
4      3.0
      ... 
873    3.0
874    3.0
875    3.0
876    3.0
877    3.0
Name: garage_cond, Length: 871, dtype: float64

In [39]:
df_test['fence'] = df_test['fence'].replace({
    'GdPrv' : 4,
    'MnPrv': 3,
    'GdWo': 2,
    'MnWw': 1,
    'NA': 0
})
df_test['fence']

0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
      ... 
873    NaN
874    3.0
875    NaN
876    2.0
877    2.0
Name: fence, Length: 871, dtype: float64

In [40]:
df_test['street'] = df_test['street'].replace({
    'Grvl' : 1,
    'Pave': 0,
})
df_test['street']

0      0
1      0
2      0
3      0
4      0
      ..
873    0
874    0
875    0
876    0
877    0
Name: street, Length: 871, dtype: int64

In [41]:
df_test['lot_shape'] = df_test['lot_shape'].replace({
    'Reg' : 4,
    'IR1': 3,
    'IR2': 2,
    'IR3': 1,
})
df_test['lot_shape']

0      4
1      3
2      3
3      4
4      3
      ..
873    4
874    4
875    4
876    4
877    4
Name: lot_shape, Length: 871, dtype: int64

In [42]:
df_test['land_contour'] = df_test['land_contour'].replace({
    'Lvl' : 4,
    'Bnk': 3,
    'HLS': 2,
    'Low': 1,
})
df_test['land_contour']

0      4
1      4
2      4
3      4
4      4
      ..
873    4
874    4
875    4
876    4
877    4
Name: land_contour, Length: 871, dtype: int64

In [43]:
df_test['utilities'] = df_test['utilities'].replace({
    'AllPub' : 4,
    'NoSewr': 3,
    'NoSeWa': 2,
    'ELO': 1,
})
df_test['utilities']

0      4
1      4
2      4
3      4
4      4
      ..
873    4
874    4
875    4
876    4
877    4
Name: utilities, Length: 871, dtype: int64

In [44]:
df_test['lot_config'] = df_test['lot_config'].replace({
    'Inside' : 5,
    'Corner': 4,
    'CulDSac': 3,
    'FR2': 2,
    'FR3': 1
})
df_test['lot_config']

0      5
1      5
2      5
3      5
4      5
      ..
873    5
874    5
875    5
876    2
877    4
Name: lot_config, Length: 871, dtype: int64

In [45]:
df_test['land_slope'] = df_test['land_slope'].replace({
    'Gtl' : 1,
    'Mod': 2,
    'Sev': 3,
})
df_test['land_slope']

0      1
1      1
2      1
3      1
4      1
      ..
873    1
874    1
875    1
876    1
877    1
Name: land_slope, Length: 871, dtype: int64

In [46]:
df_test['condition_1'] = df_test['condition_1'].replace({
    'Artery' : 9,
    'Feedr': 8,
    'Norm': 7,
    'RRNn': 6,
    'RRAn': 5,
    'PosN': 4,
    'PosA': 3,
    'RRNe': 2,
    'RRAe': 1,
})
df_test['condition_1']

0      7
1      7
2      7
3      7
4      7
      ..
873    4
874    7
875    8
876    7
877    7
Name: condition_1, Length: 871, dtype: int64

In [47]:
df_test['condition_2'] = df_test['condition_2'].replace({
    'Artery' : 9,
    'Feedr': 8,
    'Norm': 7,
    'RRNn': 6,
    'RRAn': 5,
    'PosN': 4,
    'PosA': 3,
    'RRNe': 2,
    'RRAe': 1,
})
df_test['condition_2']

0      7
1      7
2      7
3      7
4      7
      ..
873    7
874    7
875    7
876    7
877    7
Name: condition_2, Length: 871, dtype: int64

In [48]:
df_test['bldg_type'] = df_test['bldg_type'].replace({
    '1Fam' : 1,
    '2fmCon': 2,
    'Duplex': 3,
    'Twnhs': 4,
    'TwnhsE': 5,
    'TwnhsT': 6
})
df_test['bldg_type']

0      2
1      3
2      1
3      1
4      1
      ..
873    1
874    1
875    1
876    1
877    1
Name: bldg_type, Length: 871, dtype: int64

In [49]:
df_test['house_style'] = df_test['house_style'].replace({
    '1Story' : 1,
    '1.5Fin': 1.5,
    '1.5Unf': 1.25,
    '2Story': 2,
    '2.5Fin': 2.5,
    '2.5Unf': 2.25,
    'SFoyer': 3,
    'SLvl': 4
})
df_test['house_style']

0      2.0
1      1.0
2      2.0
3      1.0
4      1.0
      ... 
873    2.0
874    2.0
875    1.0
876    1.0
877    1.0
Name: house_style, Length: 871, dtype: float64

In [50]:
df_test['roof_style'] = df_test['roof_style'].replace({
    'Flat' : 1,
    'Gable': 2,
    'Gambrel': 3,
    'Hip': 4,
    'Mansard': 5,
    'Shed': 6
})
df_test['roof_style']

0      2
1      2
2      2
3      2
4      2
      ..
873    2
874    2
875    4
876    2
877    2
Name: roof_style, Length: 871, dtype: int64

In [51]:
df_test['roof_matl'] = df_test['roof_matl'].replace({
    'ClyTile' : 9,
    'CompShg': 8,
    'Membran': 7,
    'Hip': 6,
    'Metal': 5,
    'Roll': 4,
    'Tar&Grv': 3,
    'WdShake': 2,
    'WdShngl': 1
})
df_test['roof_matl']

0      8
1      8
2      8
3      8
4      8
      ..
873    8
874    8
875    8
876    8
877    8
Name: roof_matl, Length: 871, dtype: int64

In [52]:
df_test['exterior_1st'] = df_test['exterior_1st'].replace({
    'AsbShng' : 17,
    'AsphShn': 16,
    'BrkComm': 15,
    'BrkFace': 14,
    'CBlock': 13,
    'CemntBd': 12,
    'HdBoard': 11,
    'ImStucc': 10,
    'MetalSd': 9,
    'Other': 8,
    'Plywood': 7,
    'PreCast': 6,
    'Stone': 5,
    'Stucco': 4,
    'VinylSd': 3,
    'Wd Sdng': 2,
    'WdShing': 1,
})
df_test['exterior_1st']

0      17
1       7
2       3
3       2
4       7
       ..
873    11
874     3
875    11
876    11
877     9
Name: exterior_1st, Length: 871, dtype: int64

In [53]:
df_test['exterior_2nd'] = df_test['exterior_2nd'].replace({
    'AsbShng' : 17,
    'AsphShn': 16,
    'Brk Cmn': 15,
    'BrkFace': 14,
    'CBlock': 13,
    'CmentBd': 12,
    'HdBoard': 11,
    'ImStucc': 10,
    'MetalSd': 9,
    'Other': 8,
    'Plywood': 7,
    'PreCast': 6,
    'Stone': 5,
    'Stucco': 4,
    'VinylSd': 3,
    'Wd Sdng': 2,
    'Wd Shng': 1
})
df_test['exterior_2nd']

#note that had error in WdShing (notation differ than exterior_1st)

0      17
1       7
2       3
3       2
4       7
       ..
873    11
874     3
875    11
876    11
877     9
Name: exterior_2nd, Length: 871, dtype: int64

In [54]:
df_test['mas_vnr_type'] = df_test['mas_vnr_type'].replace({
    'BrkCmn' : 4,
    'BrkFace': 3,
    'CBlock': 2,
    'None': 0,
    'Stone': 1,
})
df_test['mas_vnr_type']

0      0.0
1      0.0
2      0.0
3      0.0
4      3.0
      ... 
873    0.0
874    3.0
875    0.0
876    0.0
877    0.0
Name: mas_vnr_type, Length: 871, dtype: float64

In [55]:
df_test['electrical'] = df_test['electrical'].replace({
    'SBrkr' : 5,
    'FuseA': 4,
    'FuseF': 3,
    'FuseP': 2,
    'Mix': 1,
})
df_test['electrical']

0      2.0
1      5.0
2      5.0
3      5.0
4      5.0
      ... 
873    5.0
874    5.0
875    5.0
876    5.0
877    5.0
Name: electrical, Length: 871, dtype: float64

In [56]:
df_test['functional'] = df_test['functional'].replace({
    'Typ' : 8,
    'Min1': 7,
    'Min2': 6,
    'Mod': 5,
    'Maj1': 4,
    'Maj2': 3,
    'Sev': 2,
    'Sal': 1
})
df_test['functional']

0      8
1      8
2      8
3      8
4      8
      ..
873    8
874    8
875    8
876    8
877    8
Name: functional, Length: 871, dtype: int64

In [57]:
df_test['garage_type'] = df_test['garage_type'].replace({
    '2Types' : 6,
    'Attchd': 5,
    'Basment': 4,
    'BuiltIn': 3,
    'CarPort': 2,
    'Detchd': 1,
    'NA': 0
})
df_test['garage_type']

0      1.0
1      5.0
2      5.0
3      1.0
4      5.0
      ... 
873    5.0
874    5.0
875    5.0
876    1.0
877    5.0
Name: garage_type, Length: 871, dtype: float64

In [58]:
df_test['garage_finish'] = df_test['garage_finish'].replace({
    'Fin' : 3,
    'RFn': 2,
    'Unf': 1,
    'NA': 0
})
df_test['garage_finish']

0      1.0
1      3.0
2      2.0
3      1.0
4      2.0
      ... 
873    1.0
874    2.0
875    1.0
876    1.0
877    1.0
Name: garage_finish, Length: 871, dtype: float64

In [59]:
df_test['paved_drive'] = df_test['paved_drive'].replace({
    'Y' : 3,
    'P': 2,
    'N': 1,
})
df_test['paved_drive']

0      3
1      3
2      3
3      1
4      3
      ..
873    3
874    3
875    3
876    3
877    3
Name: paved_drive, Length: 871, dtype: int64

In [61]:
df_test['sale_type'] = df_test['sale_type'].replace({
    'WD ': 10, 
    'CWD': 9,
    'VWD': 8,
    'New': 7,
    'COD': 6,
    'Con': 5,
    'ConLw': 4,
    'ConLI': 3,
    'ConLD': 2,
    'Oth': 1, 
})
df_test['sale_type']

0      10
1      10
2       7
3      10
4      10
       ..
873    10
874    10
875    10
876    10
877    10
Name: sale_type, Length: 871, dtype: int64

In [62]:
df_test['foundation'] = df_test['foundation'].replace({
    'BrkTil': 6,
    'CBlock': 5,
    'PConc': 4,
    'Slab': 3, 
    'Stone': 2, 
    'Wood': 1,
})
df_test['foundation']

0      2
1      5
2      4
3      5
4      5
      ..
873    5
874    5
875    5
876    4
877    5
Name: foundation, Length: 871, dtype: int64

In [63]:
df_test['neighborhood'] = df_test['neighborhood'].replace({
    'Blmngtn': 28, 
    'Blueste': 27,
    'BrDale': 26,
    'BrkSide': 25,
    'ClearCr': 24,
    'CollgCr': 23,
    'Crawfor': 22,
    'Edwards': 21,
    'Gilbert': 20,
    'Greens': 19,
    'GrnHill': 18,
    'IDOTRR': 17,
    'Landmrk': 16,
    'MeadowV': 15,
    'Mitchel': 14,
    'Names': 13,
    'NoRidge': 12,
    'NPkVill': 11,
    'NridgHt': 10,
    'NAmes': 9,
    'NWAmes': 9,
    'OldTown': 8,
    'SWISU': 7,
    'Sawyer': 6,
    'SawyerW': 5,
    'Somerst': 4,
    'StoneBr': 3,
    'Timber': 2,
    'Veenker': 1,
})
df_test['neighborhood']
# note here 'NAmes' = 'NWAmes' = 9

0       8
1       6
2      20
3       8
4       9
       ..
873     9
874     9
875     6
876     9
877     9
Name: neighborhood, Length: 871, dtype: int64

In [64]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 871 entries, 0 to 877
Data columns (total 77 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               871 non-null    int64  
 1   pid              871 non-null    int64  
 2   ms_subclass      871 non-null    int64  
 3   ms_zoning        871 non-null    int64  
 4   lot_frontage     714 non-null    float64
 5   lot_area         871 non-null    int64  
 6   street           871 non-null    int64  
 7   lot_shape        871 non-null    int64  
 8   land_contour     871 non-null    int64  
 9   utilities        871 non-null    int64  
 10  lot_config       871 non-null    int64  
 11  land_slope       871 non-null    int64  
 12  neighborhood     871 non-null    int64  
 13  condition_1      871 non-null    int64  
 14  condition_2      871 non-null    int64  
 15  bldg_type        871 non-null    int64  
 16  house_style      871 non-null    float64
 17  overall_qual    

In [80]:
#set these rows to 0
df_test.loc[(df_test['lot_frontage'].isnull()) & (df_test['ms_zoning']==3), ['lot_frontage']].replace(np.nan, 0)
df_test.loc[(df_test['lot_frontage'].isnull()) & (df_test['ms_zoning']!=3), :]
df_test['lot_frontage'] = df_test['lot_frontage'].fillna(0)
df_test['fence'] = df_test['fence'].fillna(0)
df_test['bsmtfin_type_2'] = df_test['bsmtfin_type_2'].fillna(0)
df_test['fireplace_qu'] = df_test['fireplace_qu'].fillna(0)
df_test['garage_type'] = df_test['garage_type'].fillna(0)
df_test['garage_yr_blt'] = df_test['garage_yr_blt'].fillna(0)
df_test['garage_finish'] = df_test['garage_finish'].fillna(0)
df_test['garage_qual'] = df_test['garage_qual'].fillna(0)
df_test['garage_cond'] = df_test['garage_cond'].fillna(0)

In [81]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 871 entries, 0 to 877
Data columns (total 77 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               871 non-null    int64  
 1   pid              871 non-null    int64  
 2   ms_subclass      871 non-null    int64  
 3   ms_zoning        871 non-null    int64  
 4   lot_frontage     871 non-null    float64
 5   lot_area         871 non-null    int64  
 6   street           871 non-null    int64  
 7   lot_shape        871 non-null    int64  
 8   land_contour     871 non-null    int64  
 9   utilities        871 non-null    int64  
 10  lot_config       871 non-null    int64  
 11  land_slope       871 non-null    int64  
 12  neighborhood     871 non-null    int64  
 13  condition_1      871 non-null    int64  
 14  condition_2      871 non-null    int64  
 15  bldg_type        871 non-null    int64  
 16  house_style      871 non-null    float64
 17  overall_qual    

In [84]:
# drop columns where they are nulls, with small # of rows
df_test = df_test.drop(df_test[df_test['mas_vnr_type'].isnull()].index)
df_test = df_test.drop(df_test[df_test['bsmt_exposure'].isnull()].index)

In [85]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 845 entries, 0 to 877
Data columns (total 77 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               845 non-null    int64  
 1   pid              845 non-null    int64  
 2   ms_subclass      845 non-null    int64  
 3   ms_zoning        845 non-null    int64  
 4   lot_frontage     845 non-null    float64
 5   lot_area         845 non-null    int64  
 6   street           845 non-null    int64  
 7   lot_shape        845 non-null    int64  
 8   land_contour     845 non-null    int64  
 9   utilities        845 non-null    int64  
 10  lot_config       845 non-null    int64  
 11  land_slope       845 non-null    int64  
 12  neighborhood     845 non-null    int64  
 13  condition_1      845 non-null    int64  
 14  condition_2      845 non-null    int64  
 15  bldg_type        845 non-null    int64  
 16  house_style      845 non-null    float64
 17  overall_qual    

In [86]:
#df_test.to_csv('datasets/test_clean.csv', index=False)

In [95]:
df_test['saleprice'] = ""

In [98]:
df_test

,id,pid,ms_subclass,ms_zoning,lot_frontage,lot_area,street,lot_shape,land_contour,utilities,...,enclosed_porch,3ssn_porch,screen_porch,pool_area,fence,misc_val,mo_sold,yr_sold,sale_type,saleprice
0,2658,902301120,190,2,69.0,9142,0,4,4,4,...,112,0,0,0,0.0,0,4,2006,10,
1,2718,905108090,90,3,0.0,9662,0,3,4,4,...,0,0,0,0,0.0,0,8,2006,10,
2,2414,528218130,60,3,58.0,17104,0,3,4,4,...,0,0,0,0,0.0,0,9,2006,7,
3,1989,902207150,30,2,60.0,8520,0,4,4,4,...,184,0,0,0,0.0,0,7,2007,10,
4,625,535105100,20,3,0.0,9500,0,3,4,4,...,0,0,185,0,0.0,0,7,2009,10,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
873,1662,527377110,60,3,80.0,8000,0,4,4,4,...,0,0,0,0,0.0,0,11,2007,10,
874,1234,535126140,60,3,90.0,14670,0,4,4,4,...,0,0,0,0,3.0,0,8,2008,10,
875,1373,904100040,20,3,55.0,8250,0,4,4,4,...,0,0,0,0,0.0,0,8,2008,10,
876,1672,527425140,20,3,60.0,9000,0,4,4,4,...,0,0,0,0,2.0,0,5,2007,10,


## Model Prep
____

In [100]:
df_test['garage_cars*kitchen_qual'] = df_test['garage_cars'] * df_test['kitchen_qual']
df_test['garage_cars*gr_liv_area'] = df_test['garage_cars'] * df_test['gr_liv_area']
df_test.head()

,id,pid,ms_subclass,ms_zoning,lot_frontage,lot_area,street,lot_shape,land_contour,utilities,...,screen_porch,pool_area,fence,misc_val,mo_sold,yr_sold,sale_type,saleprice,garage_cars*kitchen_qual,garage_cars*gr_liv_area
0,2658,902301120,190,2,69.0,9142,0,4,4,4,...,0,0,0.0,0,4,2006,10,,2,1928
1,2718,905108090,90,3,0.0,9662,0,3,4,4,...,0,0,0.0,0,8,2006,10,,6,3934
2,2414,528218130,60,3,58.0,17104,0,3,4,4,...,0,0,0.0,0,9,2006,7,,8,2992
3,1989,902207150,30,2,60.0,8520,0,4,4,4,...,0,0,0.0,0,7,2007,10,,6,1936
4,625,535105100,20,3,0.0,9500,0,3,4,4,...,185,0,0.0,0,7,2009,10,,6,2788


In [128]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 845 entries, 0 to 877
Data columns (total 80 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   id                        845 non-null    int64  
 1   pid                       845 non-null    int64  
 2   ms_subclass               845 non-null    int64  
 3   ms_zoning                 845 non-null    int64  
 4   lot_frontage              845 non-null    float64
 5   lot_area                  845 non-null    int64  
 6   street                    845 non-null    int64  
 7   lot_shape                 845 non-null    int64  
 8   land_contour              845 non-null    int64  
 9   utilities                 845 non-null    int64  
 10  lot_config                845 non-null    int64  
 11  land_slope                845 non-null    int64  
 12  neighborhood              845 non-null    int64  
 13  condition_1               845 non-null    int64  
 14  condition_

In [141]:
X = df_test[['mas_vnr_area','totrms_abvgrd', 'fireplace_qu', 'year_remod/add', 'full_bath', 'garage_finish',
        'year_built', 'total_bsmt_sf', '1st_flr_sf', 'garage_cars', 'bsmt_qual', 'kitchen_qual',
        'gr_liv_area', 'exter_qual', 'overall_qual', 'garage_cars*kitchen_qual', 'garage_cars*gr_liv_area']]
y = df_test['saleprice']

In [142]:
# put the X and y into a DataFrame
df_test2 = pd.DataFrame(df_test, columns=['mas_vnr_area','totrms_abvgrd', 'fireplace_qu', 'year_remod/add', 'full_bath', 'garage_finish',
        'year_built', 'total_bsmt_sf', '1st_flr_sf', 'garage_cars', 'bsmt_qual', 'kitchen_qual',
        'gr_liv_area', 'exter_qual', 'overall_qual', 'garage_cars*kitchen_qual', 'garage_cars*gr_liv_area', 
                                            'saleprice'])
df_test2

,mas_vnr_area,totrms_abvgrd,fireplace_qu,year_remod/add,full_bath,garage_finish,year_built,total_bsmt_sf,1st_flr_sf,garage_cars,bsmt_qual,kitchen_qual,gr_liv_area,exter_qual,overall_qual,garage_cars*kitchen_qual,garage_cars*gr_liv_area,saleprice
0,0.0,9,0.0,1950,2,1.0,1910,1020,908,1,2.0,2,1928,3,6,2,1928,
1,0.0,10,0.0,1977,2,3.0,1977,1967,1967,2,4.0,3,1967,3,5,6,3934,
2,0.0,7,4.0,2006,2,2.0,2006,654,664,2,4.0,4,1496,4,7,8,2992,
3,0.0,5,0.0,2006,1,1.0,1923,968,968,2,3.0,3,968,4,5,6,1936,
4,247.0,6,4.0,1963,1,2.0,1963,1394,1394,2,4.0,3,1394,3,6,6,2788,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
873,0.0,8,3.0,1974,2,1.0,1974,1084,1084,2,3.0,3,1877,3,6,6,3754,
874,410.0,9,4.0,1999,2,2.0,1966,1104,1104,2,3.0,4,1988,4,6,8,3976,
875,0.0,5,3.0,1968,1,1.0,1968,952,1211,1,3.0,3,1211,3,5,3,1211,
876,0.0,5,0.0,1971,1,1.0,1971,864,864,2,3.0,3,864,3,4,6,1728,


In [139]:
df_test2.to_csv('datasets/test_clean2.csv', index=False)

In [140]:
df_test2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 845 entries, 0 to 877
Data columns (total 18 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   mas_vnr_area              845 non-null    float64
 1   totrms_abvgrd             845 non-null    int64  
 2   fireplace_qu              845 non-null    float64
 3   year_remod/add            845 non-null    int64  
 4   full_bath                 845 non-null    int64  
 5   garage_finish             845 non-null    float64
 6   year_built                845 non-null    int64  
 7   total_bsmt_sf             845 non-null    int64  
 8   1st_flr_sf                845 non-null    int64  
 9   garage_cars               845 non-null    int64  
 10  bsmt_qual                 845 non-null    float64
 11  kitchen_qual              845 non-null    int64  
 12  gr_liv_area               845 non-null    int64  
 13  exter_qual                845 non-null    int64  
 14  overall_qu